<a href="https://colab.research.google.com/github/NotAbdelrahmanelsayed/Hands-on-machine-learning/blob/main/Ensemble_learning_and_RandomForests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

**how ensemble learning works !**

In [2]:
#import the moons data first
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
svm_clf = SVC()
rnd_clf = RandomForestClassifier()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('svc', svm_clf), ('rf', rnd_clf)],
)

**let's look at the accuarecy**

In [4]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, svm_clf, rnd_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
SVC 0.896
RandomForestClassifier 0.888
VotingClassifier 0.904


**The voting classifier is slightly better than the best classifier**

Random forest

In [5]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=6, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92

#AdaBoostClassifier 
**is a classifier try many random forset algorithms and choose the best i.e 100 algorithm and choose the best one or stop when he found the local optimum algorith let's take a look** 

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators = 300, 
    learning_rate = 0.5
)
ada_clf.fit(X_train, y_train)


ada_clf.score(X_test, y_test)


0.896

**Gradient Boosting**


working for classification and regression just like adaboost


Let create a simple quadratic dataset:

}

In [7]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [8]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2) # max_depth is the maximum leaves of the tree if none it maybe ovefit the data
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [9]:
y2 = y - tree_reg.predict(X)
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [10]:
y3 = y2 - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [11]:
X_new = np.array([[0.8]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg))


In [12]:
y_pred

array([0.75026781])

**every single tree are trained of the residuals from previous tree and so on till we get the best predections**


**we can use early stopping while find the iptimal number of trees , using GBRT**

In [13]:
import numpy as np 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120) # (n_estimators)try to find the local optimal number of trees in 120 try

gbrt.fit(X,y)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [14]:
errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
best_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=best_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

# Excercise Time .

Exercise: Load the MNIST data and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing).



In [15]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version = 1 , as_frame = False)

X = mnist['data']
y = mnist['target'].astype(np.uint8)

X_train = X[:60000]
y_train = y[:60000]
X_test  = X[60000:]
y_test = y[60000:]

Exercise: Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM.



In [29]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier


svc_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
randomforest_clf = RandomForestClassifier(n_estimators=200, random_state=42)
extratree_clf = ExtraTreesClassifier(n_estimators=200, random_state=42)


In [30]:
estimators = [svc_clf, randomforest_clf, extratree_clf]

for estimator in estimators:
  estimator.fit(X_train, y_train)

In [31]:
[estimator.score(X_test, y_test) for estimator in estimators]

[0.865, 0.9707, 0.9735]

Exercise: Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier.



In [32]:
voting_estimators = [('svc_clf', svc_clf), ('randomforest_clf', randomforest_clf), ('extratree_clf',extratree_clf)]
voting_clf = VotingClassifier(voting_estimators, voting='hard')
voting_clf.fit(X_train,y_train)
voting_clf.score(X_test,y_test)

0.9714

**lets try the soft voting**

In [36]:
voting_clf.voting = "soft"
voting_clf.score(X_test,y_test)


AttributeError: ignored

**svc have a bad predection i will delete it and fix the error**

In [38]:
voting_clf.estimators_
del voting_clf.estimators_[0]

In [39]:
voting_clf.voting = "soft"
voting_clf.score(X_test,y_test)


0.9734

In [40]:
voting_clf.voting = "hard"
voting_clf.score(X_test,y_test)


0.9721

**soft voting wins !**

# New Section